In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py
from functions import process_network_summary

%matplotlib inline
from IPython.display import display, HTML

pd.options.display.float_format = '{:0,.0f}'.format

In [2]:
df = process_network_summary()

# Truck VMT by County


In [3]:

df['medium_truck_vmt'] = df['@mveh']*df['length']
df['heavy_truck_vmt'] = df['@hveh']*df['length']
_df = df[['medium_truck_vmt','heavy_truck_vmt','county']].groupby('county').sum()[['medium_truck_vmt','heavy_truck_vmt']]
_df.loc['Total',:] = _df.sum()
_df = _df.reset_index()
_df.rename(columns={'county':'County', 'medium_truck_vmt': 'Medium Trucks', 'heavy_truck_vmt': 'Heavy Trucks'}, inplace=True)
_df


,County,Medium Trucks,Heavy Trucks
0,King,"1,956,019","1,417,345"
1,Kitsap,"145,016","90,382"
2,Outside Region,21,"22,280"
3,Pierce,"549,340","539,811"
4,Snohomish,"551,392","534,327"
5,Total,"3,201,788","2,604,147"


# Congestion by County

In [4]:
df['Medium and Heavy Trucks'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"614,182","56,178","5,619","232,892","235,207"
Moderate,"138,760",674,NaN,"33,508","24,960"
Heavy,"88,887","1,219",NaN,"6,393","11,572"
Severe,"5,637",47,0,466,456
Total,"847,465","58,118","5,619","273,259","272,195"


In [5]:
_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"352,832","32,838","3,301","135,995","134,757"
Moderate,"97,149",650,1,"19,533","19,552"
Heavy,"44,397",639,NaN,"4,371","5,108"
Severe,"2,936",45,0,324,305
Total,"497,314","34,171","3,302","160,222","159,722"


# Truck Miles on FGTS
Freight and Goods Transportation System
- T-1: More than 10 million tons per year
- T-2: 2 4 million to 10 million tons per year

In [6]:
pd.options.display.float_format = '{:0,.0f}'.format
df['Non-Truck'] = df['VMT']-df['Medium and Heavy Trucks']
_df = df[['medium_truck_vmt','heavy_truck_vmt','@fgts','Non-Truck']].melt(
    id_vars='@fgts', var_name='Vehicle Type')
_df = _df.pivot_table(index='Vehicle Type', columns='@fgts', 
                values='value', aggfunc='sum')
# _df = _df.reset_index()
_df = _df.rename_axis(None, axis=1)
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df.rename(index={'heavy_truck_vmt': 'Heavy Trucks',
                 'medium_truck_vmt': 'Medium Trucks'}, inplace=True)
_df[['T-1','T-2','Other Routes']]

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,"31,108,580","11,392,786","34,021,455"
Heavy Trucks,"2,086,616","171,753","345,778"
Medium Trucks,"1,778,553","486,250","936,984"


In [7]:
pd.options.display.float_format = '{:0,.1%}'.format
_df[['T-1','T-2','Other Routes']]/_df[['T-1','T-2','Other Routes']].sum()

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,88.9%,94.5%,96.4%
Heavy Trucks,6.0%,1.4%,1.0%
Medium Trucks,5.1%,4.0%,2.7%


# Congestion on FGTS

In [8]:
pd.options.display.float_format = '{:0,.0f}'.format

# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@mveh']+df['@hveh']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"2,187,416","833,209","1,766,699"
Moderate,"449,411","105,827","211,493"
Heavy,"267,387","93,124","191,655"
Severe,"29,488","34,847","80,162"
Total,"2,933,702","1,067,007","2,250,009"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,78.5%,74.6%,78.1%
Moderate,9.4%,15.3%,9.9%
Heavy,8.5%,9.1%,8.7%
Severe,3.6%,1.0%,3.3%


In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@hveh']+df['@mveh']

_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"1,246,204","484,373","995,571"
Moderate,"305,609","60,678","130,523"
Heavy,"142,907","51,872","117,576"
Severe,"18,279","24,044","48,839"
Total,"1,712,998","620,967","1,292,509"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,77.0%,72.7%,78.0%
Moderate,10.1%,17.8%,9.8%
Heavy,9.1%,8.3%,8.4%
Severe,3.8%,1.1%,3.9%


# Equity

## Households Within 500' of Heavy Truck Volumes
Total number of households within 500' of T-1 and T-2 routes
- T-1: More than 10 million tons per year
- T-2: 2 4 million to 10 million tons per year

In [12]:
import polars as pl

# Intersect buffer with land use file
df_lu = pl.read_csv(r'..\..\..\..\outputs\landuse\parcels_urbansim.txt', separator=' ').to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))
# Set the coordinate reference system (CRS) to match the land use data
gdf_lu.crs = 'EPSG:2285'

# Buffer the parcels at 500ft
gdf_lu['geometry'] = gdf_lu.buffer(500)

In [13]:

# Intersect this geography  with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
# Do not include connectors since these are abstracted ul3==5; also remove weave links ul3==0 
gdf_network = gdf_network[~gdf_network.ul3.isin([0,5])]
# Truck network links are those that are in FGTS 1 or 2 system
gdf_network = gdf_network[gdf_network['FGTS'].isin([1,2])]

gdf_intersect = gpd.overlay(gdf_network, gdf_lu, how="intersection", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

In [14]:
truck_parcels = gdf_intersect.groupby('parcelid').first()[['hh_p']].reset_index()


# Result should be the network components with some flags for the parcelid
# We can take the parcel information, join with parcel info and group
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../../configuration/input_configuration.toml'))

parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])
df = truck_parcels.merge(parcel_geog,left_on='parcelid', right_on='ParcelID')

In [15]:
# Get the total number of households that in equtiy geograhpies
# Comprae the percent of those that are in the buffer versus those that are not
# For the 4 equity groups, perform the calc and add as a table
results_df = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  }.items():
    _df = df[[col,'hh_p']].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df = pd.concat([results_df,_df])
results_df = results_df.reset_index()
results_df = results_df[results_df['index']>=0]

## Total Households Within 500' of T-1/T-2 Routes by Equity Group

In [16]:
_df = results_df.pivot_table(index='index', columns='equity_group', values='hh_p', aggfunc='sum')
_df.index = _df.index.astype('int').map({0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )
pd.options.display.float_format = '{:0,.0f}'.format
_df.loc['Region',:] = _df.sum(axis=0)
_df.astype('float')

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,"96,729","105,618","118,343","75,091","91,169","124,528"
Above Regional Average,"56,908","41,623","53,783","65,912","56,922","45,724"
Higher Share of Equity Population,"33,259","39,655","14,770","45,893","38,805","16,644"
Region,"186,896","186,896","186,896","186,896","186,896","186,896"


In [17]:
pd.options.display.float_format = '{:0,.1%}'.format
_df.drop('Region')/_df.drop('Region').sum()

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,51.8%,56.5%,63.3%,40.2%,48.8%,66.6%
Above Regional Average,30.4%,22.3%,28.8%,35.3%,30.5%,24.5%
Higher Share of Equity Population,17.8%,21.2%,7.9%,24.6%,20.8%,8.9%


In [18]:
_df = results_df[["equity_group","hh_p","index"]].pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_buffer = _df.copy()

df_lu_tot = df_lu[['parcelid','hh_p']].merge(parcel_geog,left_on='parcelid', right_on='ParcelID')
df_lu_tot['Region'] = 1
results_df_tot = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  'Region': 'Region'
                  }.items():
    _df = df_lu_tot[['hh_p',col]].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df_tot = pd.concat([results_df_tot, _df])
results_df_tot = results_df_tot.reset_index()
pd.options.display.float_format = '{:0,.0f}'.format

_df = results_df_tot.pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_tot = _df.copy()
_df = _df_tot.merge(_df_buffer, left_index=True, right_index=True, suffixes=['_tot','_buffer'])

pd.options.display.float_format = '{:0,.1%}'.format
efa_list = ['Below Regional Average', 'Above Regional Average','Higher Share of Equity Population', 'Total Households']
for efa in efa_list:
    _df[efa] = _df[f'{efa}_buffer']/_df[f'{efa}_tot']
_df[efa_list].T


,Disability,LEP,Older,People of Color,Poverty,Youth
Below Regional Average,10.3%,9.7%,13.0%,8.0%,8.8%,13.1%
Above Regional Average,10.8%,11.7%,9.1%,13.2%,12.8%,8.1%
Higher Share of Equity Population,12.5%,13.8%,6.3%,15.7%,15.6%,7.7%
Total Households,10.8%,10.8%,10.8%,10.8%,10.8%,10.8%
